In [1]:
import pandas as pd
import numpy
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [2]:
df = pd.read_csv("C:/Users/Elvita/Desktop/data.csv", low_memory=False)
len(df)

4455

In [3]:
df.drop(df[df['ACCOUNT_STATUS'] !='A'].index, inplace = True)
df.reset_index(inplace = True)

In [4]:
len(df)

4409

In [5]:
acc_no=df.Account_No.unique()
acc_no=acc_no.tolist()

tx_code=df.TRAN_PARTICULAR_CODE.unique()
tx_code=tx_code.tolist()

In [6]:
for i in range(len(tx_code)-1):
    if pd.isnull(tx_code[i])==True:
        tx_code.remove(tx_code[i])

In [7]:
len(acc_no)

102

In [8]:
column_names=['count_C','count_D','sum_C','sum_D','C-D','Null_C','Null_D','UPI_C','UPI_D']
column_names.extend(tx_code)

In [9]:
df1=pd.DataFrame(columns=column_names)

In [11]:
c=0
for i in range(len(acc_no)):
    count_c,count_d,sum_c,sum_d,null_c,null_d,diff,upi_c,upi_d=0,0,0,0,0,0,0,0,0
    for j in range(len(df)):      
        if df.loc[j,'Account_No']==acc_no[i]:
            if df.loc[j,'DEBIT_CREDIT_INDICATOR']=='C':
                count_c+=1
                sum_c+=df.loc[j,'TRAN_AMOUNT']
                if pd.isnull(df.loc[j,'TRAN_PARTICULAR_CODE'])==True:
                    null_c+=1
                if df.loc[j,'TRAN_PARTICULAR_CODE']=='UPI':
                    upi_c+=1
        if df.loc[j,'Account_No']==acc_no[i]:
            if df.loc[j,'DEBIT_CREDIT_INDICATOR']=='D':
                count_d+=1
                sum_d+=df.loc[j,'TRAN_AMOUNT']
                if pd.isnull(df.loc[j,'TRAN_PARTICULAR_CODE'])==True:
                    null_d+=1
                if df.loc[j,'TRAN_PARTICULAR_CODE']=='UPI':
                    upi_d+=1
    diff=sum_c-sum_d
    input_val=[count_c,count_d,sum_c,sum_d,diff,null_c,null_d,upi_c,upi_d]
    for j in range(len(tx_code)):
        tran_code=0
        for k in range(len(df)):
            if df.loc[k,'Account_No']==acc_no[i]:
                if df.loc[k,'TRAN_PARTICULAR_CODE']==tx_code[j]:
                    tran_code+=1
        input_val.append(tran_code)    
    c+=1
    df1.loc[i] = input_val
print(c)
    

102


In [12]:
df1

,count_C,count_D,sum_C,sum_D,C-D,Null_C,Null_D,UPI_C,UPI_D,IMP,...,EBK,ACHNR,PIS,CADEP,CAWID,TAPCR,IMPR,MOBRB,TDSTR,SAL
0,80.0,234.0,704893.49,704474.91,418.58,2.0,7.0,63.0,205.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,50.0,73.0,219913.00,285566.00,-65653.00,3.0,0.0,47.0,65.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,0.0,2023.00,0.00,2023.00,2.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,56.0,306.0,722121.00,711672.06,10448.94,6.0,20.0,11.0,224.0,8.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,21.0,80.0,594253.23,561220.34,33032.89,3.0,9.0,13.0,68.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,1.0,1.0,8.79,8.79,0.00,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,1.0,1.0,6.02,6.02,0.00,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,2.0,2.0,80.52,80.52,0.00,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,4.0,4.0,85.72,85.72,0.00,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
dep_indicator=[]
for i in range(len(df1)):
    if ((df1.loc[i,'C-D']>0 or df1.loc[i,'C-D']<=0) and (((df1.loc[i,'ACMUT']>0 and df1.loc[i,'NAH']>0) or (df1.loc[i,'ACMUT']>0 and df1.loc[i,'NAH']<=0) or (df1.loc[i,'ACMUT']<=0 and df1.loc[i,'NAH']>0)))):
        dep_indicator.append('No')
    
    elif (df1.loc[i,'SAL']>0 and ((df1.loc[i,'ACMUT']>0 and df1.loc[i,'NAH']>0) or (df1.loc[i,'ACMUT']>0 and df1.loc[i,'NAH']<=0) or (df1.loc[i,'ACMUT']<=0 and df1.loc[i,'NAH']>0)) and (df1.loc[i,'POS']>20 or df1.loc[i,'POS']<=20)):
        dep_indicator.append('No')
    
    elif (df1.loc[i,'C-D']>0 and df1.loc[i,'SAL']>0 and (df1.loc[i,'TRF']>0 or df1.loc[i,'TRF']<=0)):
        dep_indicator.append('No')
        
    elif (df1.loc[i,'C-D']>0 and df1.loc[i,'SAL']>0 and (df1.loc[i,'Null_C']>0 or df1.loc[i,'UPI_D']<=70)):
        dep_indicator.append('No')
    
    elif (df1.loc[i,'C-D']>0 and df1.loc[i,'SAL']>0 and((df1.loc[i,'MOBCC']>0 or df1.loc[i,'MOBCC']<=0) and (df1.loc[i,'POS']<=20 or df1.loc[i,'POS']>20))):
        dep_indicator.append('No')
    
    elif(df1.loc[i,'C-D']<=0 and df1.loc[i,'SAL']>0  and (df1.loc[i,'TRF']>0 or df1.loc[i,'MOBCC']>0 or (df1.loc[i,'POS']>30 or df1.loc[i,'POS']<=30))):
        dep_indicator.append('Yes')
    
    elif (df1.loc[i,'C-D']>0 and df1.loc[i,'SAL']<=0 and (df1.loc[i,'ACMUT']<=0 and df1.loc[i,'NAH']<=0)):
        dep_indicator.append('Yes')
    
    elif (df1.loc[i,'SAL']>0 and df1.loc[i,'TRF']<=0 and df1.loc[i,'UPI_D']>35 and df1.loc[i,'MOBCC']>0):
        dep_indicator.append('Yes')
    
    elif ((df1.loc[i,'UPI_D']>35 and df1.loc[i,'POS']>30) and (df1.loc[i,'TRF']>0 or df1.loc[i,'TRF']<=0) and df1.loc[i,'SAL']<=0):
        dep_indicator.append('Yes')
    
    else:
        dep_indicator.append('Maybe')
df1['DEPLETION']= dep_indicator

In [74]:
from sklearn.model_selection import train_test_split
X = df1.drop(['DEPLETION'], axis = 1)
y= df1['DEPLETION']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 42)

In [67]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)

RandomForestClassifier()

In [75]:
y_pred=clf.predict(X_test)

In [78]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9607843137254902


In [80]:
metrics.confusion_matrix(y_test, y_pred)

array([[42,  0,  0],
       [ 3,  4,  0],
       [ 1,  0, 52]], dtype=int64)

In [72]:
pd.DataFrame(clf.feature_importances_, index=X_train.columns).sort_values(by=0, ascending=False)

,0
C-D,0.413950
count_D,0.079600
sum_D,0.075541
count_C,0.044331
sum_C,0.042278
ACMUT,0.039754
Null_C,0.030133
Null_D,0.024970
NEFT,0.023448
UPI_D,0.018574


In [82]:
print (clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

1.0
0.9607843137254902


test on raw data

In [ ]:
#reading and cleaning raw data
df = pd.read_csv("C:/Users/Elvita/Desktop/data.csv", low_memory=False)

df.drop(df[df['ACCOUNT_STATUS'] !='A'].index, inplace = True)
df.reset_index(inplace = True)

acc_no=df.Account_No.unique()
acc_no=acc_no.tolist()

tx_code=df.TRAN_PARTICULAR_CODE.unique()
tx_code=tx_code.tolist()

for i in range(len(tx_code)-1):
    if pd.isnull(tx_code[i])==True:
        tx_code.remove(tx_code[i])
        
column_names=['count_C','count_D','sum_C','sum_D','C-D','Null_C','Null_D','UPI_C','UPI_D']
column_names.extend(tx_code)

df1=pd.DataFrame(columns=column_names)



In [ ]:
for i in range(len(acc_no)):
    count_c,count_d,sum_c,sum_d,null_c,null_d,diff,upi_c,upi_d=0,0,0,0,0,0,0,0,0
    for j in range(len(df)):      
        if df.loc[j,'Account_No']==acc_no[i]:
            if df.loc[j,'DEBIT_CREDIT_INDICATOR']=='C':
                count_c+=1
                sum_c+=df.loc[j,'TRAN_AMOUNT']
                if pd.isnull(df.loc[j,'TRAN_PARTICULAR_CODE'])==True:
                    null_c+=1
                if df.loc[j,'TRAN_PARTICULAR_CODE']=='UPI':
                    upi_c+=1
        if df.loc[j,'Account_No']==acc_no[i]:
            if df.loc[j,'DEBIT_CREDIT_INDICATOR']=='D':
                count_d+=1
                sum_d+=df.loc[j,'TRAN_AMOUNT']
                if pd.isnull(df.loc[j,'TRAN_PARTICULAR_CODE'])==True:
                    null_d+=1
                if df.loc[j,'TRAN_PARTICULAR_CODE']=='UPI':
                    upi_d+=1
    diff=sum_c-sum_d
    input_val=[count_c,count_d,sum_c,sum_d,diff,null_c,null_d,upi_c,upi_d]
    for j in range(len(tx_code)):
        tran_code=0
        for k in range(len(df)):
            if df.loc[k,'Account_No']==acc_no[i]:
                if df.loc[k,'TRAN_PARTICULAR_CODE']==tx_code[j]:
                    tran_code+=1
        input_val.append(tran_code)    
    df1.loc[i] = input_val



In [ ]:
#logic for depletion or not
dep_indicator=[]
for i in range(len(df1)):
    if ((df1.loc[i,'C-D']>0 or df1.loc[i,'C-D']<=0) and (((df1.loc[i,'ACMUT']>0 and df1.loc[i,'NAH']>0) or (df1.loc[i,'ACMUT']>0 and df1.loc[i,'NAH']<=0) or (df1.loc[i,'ACMUT']<=0 and df1.loc[i,'NAH']>0)))):
        dep_indicator.append('No')
    
    elif (df1.loc[i,'SAL']>0 and ((df1.loc[i,'ACMUT']>0 and df1.loc[i,'NAH']>0) or (df1.loc[i,'ACMUT']>0 and df1.loc[i,'NAH']<=0) or (df1.loc[i,'ACMUT']<=0 and df1.loc[i,'NAH']>0)) and (df1.loc[i,'POS']>20 or df1.loc[i,'POS']<=20)):
        dep_indicator.append('No')
    
    elif (df1.loc[i,'C-D']>0 and df1.loc[i,'SAL']>0 and (df1.loc[i,'TRF']>0 or df1.loc[i,'TRF']<=0)):
        dep_indicator.append('No')
        
    elif (df1.loc[i,'C-D']>0 and df1.loc[i,'SAL']>0 and (df1.loc[i,'Null_C']>0 or df1.loc[i,'UPI_D']<=70)):
        dep_indicator.append('No')
    
    elif (df1.loc[i,'C-D']>0 and df1.loc[i,'SAL']>0 and((df1.loc[i,'MOBCC']>0 or df1.loc[i,'MOBCC']<=0) and (df1.loc[i,'POS']<=20 or df1.loc[i,'POS']>20))):
        dep_indicator.append('No')
    
    elif(df1.loc[i,'C-D']<=0 and df1.loc[i,'SAL']>0  and (df1.loc[i,'TRF']>0 or df1.loc[i,'MOBCC']>0 or (df1.loc[i,'POS']>30 or df1.loc[i,'POS']<=30))):
        dep_indicator.append('Yes')
    
    elif (df1.loc[i,'C-D']>0 and df1.loc[i,'SAL']<=0 and (df1.loc[i,'ACMUT']<=0 and df1.loc[i,'NAH']<=0)):
        dep_indicator.append('Yes')
    
    elif (df1.loc[i,'SAL']>0 and df1.loc[i,'TRF']<=0 and df1.loc[i,'UPI_D']>35 and df1.loc[i,'MOBCC']>0):
        dep_indicator.append('Yes')
    
    elif ((df1.loc[i,'UPI_D']>35 and df1.loc[i,'POS']>30) and (df1.loc[i,'TRF']>0 or df1.loc[i,'TRF']<=0) and df1.loc[i,'SAL']<=0):
        dep_indicator.append('Yes')
    
    else:
        dep_indicator.append('Maybe')
df1['DEPLETION']= dep_indicator



In [ ]:
X = df1.drop(['DEPLETION'], axis = 1)
y= df1['DEPLETION']

y_pred=clf.predict(X)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y, y_pred))

In [ ]:
metrics.confusion_matrix(y, y_pred)

In [ ]:
pd.DataFrame(clf.feature_importances_, index=X.columns).sort_values(by=0, ascending=False)